# Лабораторная работа №2

In [2]:
import codecs
from hashlib import md5
import os
import subprocess 
import re 
import time
import random
import datetime
import sys

1. Напишите скрипт, который читает текстовый файл и выводит символы
в порядке убывания частоты встречаемости в тексте. Регистр символа
не имеет значения. Программа должна учитывать только буквенные
символы (символы пунктуации, цифры и служебные символы следует
игнорировать). Проверьте работу скрипта на нескольких файлах с
текстом на английском и русском языках, сравните результаты с
таблицами, приведенными в http://wikipedia.org/wiki/Letter_frequencies.

In [2]:
alphabet_en = ''.join([chr(i) for i in range(ord('A'),ord('Z')+1)])
alphabet_ru = ''.join([chr(i) for i in range(ord('А'),ord('Я')+1)])

with codecs.open('lab2-1.txt','r',encoding = 'utf-8') as file:
    t = file.read().upper()

dict_letters_ru = {i:t.count(i) for i in alphabet_ru}
dict_letters_en = {i:t.count(i) for i in alphabet_en}
dr = dict_letters_ru
de = dict_letters_en

for i in sorted(dr.items(),key = lambda i: i[1],reverse=True):
    print(i[0],"-",round(i[1]/len(dr),2),"%")
    
for i in sorted(de.items(),key = lambda i: i[1],reverse=True):
    print(i[0],"-",round(i[1]/len(de),2),"%")

О - 4.97 %
Е - 4.47 %
А - 3.91 %
И - 3.19 %
Н - 3.12 %
Л - 3.0 %
С - 2.81 %
Т - 2.75 %
В - 2.72 %
Р - 2.56 %
К - 2.25 %
М - 2.12 %
Д - 1.66 %
Ы - 1.38 %
П - 1.22 %
Й - 1.03 %
Ь - 0.97 %
У - 0.94 %
Я - 0.94 %
Б - 0.84 %
Г - 0.78 %
З - 0.78 %
Х - 0.59 %
Ш - 0.47 %
Ю - 0.44 %
Ж - 0.41 %
Ч - 0.38 %
Ц - 0.22 %
Щ - 0.12 %
Э - 0.03 %
Ф - 0.0 %
Ъ - 0.0 %
E - 10.23 %
A - 6.73 %
T - 6.65 %
H - 6.12 %
N - 5.62 %
O - 5.0 %
D - 4.73 %
R - 4.69 %
I - 4.46 %
S - 4.0 %
L - 3.88 %
W - 1.92 %
F - 1.88 %
M - 1.65 %
Y - 1.62 %
G - 1.54 %
U - 1.54 %
B - 1.08 %
K - 1.08 %
C - 1.04 %
V - 0.62 %
P - 0.5 %
Q - 0.04 %
J - 0.0 %
X - 0.0 %
Z - 0.0 %


2. Напишите скрипт, позволяющий искать в заданной директории и в ее
подпапках файлы-дубликаты на основе сравнения контрольных сумм
(MD5). Файлы могут иметь одинаковое содержимое, но отличаться
именами. Скрипт должен вывести группы имен обнаруженных файловдубликатов

In [3]:
def MD5(FileName):
    m = md5()
    h = m.update(open(FileName, 'rb').read())
    return m.hexdigest()

In [4]:
directory = input('Введите полный путь к папке: ')

duplicate = {}
for dirs, subdirs, files in os.walk(directory):
    for name in files:
        file = os.path.join(dirs, name)
        try:
            duplicate[MD5(file)].append(file)
        except KeyError:
            duplicate[MD5(file)] = [file,]

duplicate = {k : duplicate[k] for k in duplicate.keys() if len(duplicate[k])>1}

duplicate

Введите полный путь к папке: task2


{'c20ad4d76fe97759aa27a0c99bff6710': ['task2\\1.txt', 'task2\\th\\ase.txt'],
 'aa25770fa35b8b458abd9945003a6bd0': ['task2\\123.txt', 'task2\\bah.txt'],
 '76d80224611fc919a5d54f0ff9fba446': ['task2\\2.txt',
  'task2\\sd\\(2).txt',
  'task2\\sd\\re\\1.txt']}

3. Задан путь к директории с музыкальными файлами (в названии
которых нет номеров, а только названия песен) и текстовый файл,
хранящий полный список песен с номерами и названиями в виде строк
формата «01. Freefall [6:12]». Напишите скрипт, который корректирует
имена файлов в директории на основе текста списка песен.

In [69]:
def duration(path_file):
    stdout, stderr = subprocess.Popen(['ffmpeg', '-i', path_file], stdout=subprocess.PIPE, 
                                      stderr=subprocess.STDOUT).communicate()
    matches = re.search(r"Duration:\s{1}(?P<hours>\d+?):(?P<minutes>\d+?):(?P<seconds>\d+)", 
                        str(stdout), re.DOTALL).groupdict() 
    return '{}.{}'.format(matches['minutes'],matches['seconds'])


In [71]:
%%time
folder = 'D:\\music'
files = os.walk('D:\\music').__next__()[2]
mus_list = ['{:02}. {} [{}]{}'.format(i+1,mus[:mus.rindex('.')],duration(os.path.join(folder,mus)),
                                       mus[mus.rindex('.'):]) for i,mus in enumerate(files)]
# random.shuffle(mus_list)

with open('tracks.txt','w',encoding = 'utf-8') as mus:
    mus.write('\n'.join(mus_list))


Wall time: 11.8 s


In [72]:
%%time
with open('tracks.txt','r',encoding = 'utf-8') as mus:
    mus_list = mus.read().split('\n')

folder = 'D:\\music'
[ os.rename(os.path.join(folder,f),os.path.join(folder,m)) 
 for m in mus_list for f in os.walk('D:\\music').__next__()[2] if f.lower()[:f.rindex('.')] in m.lower()]


Wall time: 378 ms


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

4) Напишите скрипт, который позволяет ввести с клавиатуры имя текстового файла, найти в нем с помощью регулярных выражений все подстроки определенного вида, в соответствии с вариантом. Например, для варианта № 1 скрипт должен вывести на экран следующее:
```
Строка 3, позиция 10 : найдено '11-05-2014'
Строка 12, позиция 2 : найдено '23-11-2014'
Строка 12, позиция 17 : найдено '23-11-2014'
```
Варианты: 
1. найдите все даты – подстроки вида «11-05-2014». 
2. *найдите все значения времени – подстроки вида «23:15:59».*
3. найдите все IPv4-адреса – подстроки вида «192.168.5.48».
4. найдите все строки вида «type x = value», где type – это тип (может принимать значение int, short или byte), х  любое слово, value – любое число.
5. найдите все номера телефонов – подстроки вида «(000)1112233» или «(000)111-22-33».
6. найдите все строки вида «x: type [N]», где type – это тип (может принимать значение int, short или byte), х – любое слово, N – любое положительное целое число.
7. найдите все «смайлы» – подстроки вида «:)», «:-)», «)))» (количество скобок может быть любым, начиная с 1).
8. найдите все логические выражения – подстроки вида «x&&y», «x&y», где х и у – любые слова. Количество пробелов может быть также любым.
9. найдите все донецкие почтовые индексы – подстроки вида «83000, Донецк» (первые 2 символа строго закреплены: «83»).
10. Найдите все полные имена директорий Windows – подстроки вида «C:\Dir\SubDir3».

In [29]:
with open('task4.txt','r') as f:
    text = f.read().split('\n')
print(text)

find = ["Строка {}, позиция {} : найдено '{}'".format(j, m.start(), m.group(0)) for j,t in enumerate(text) 
        for m in re.finditer(r"(([0-1]\d|[2][0-3]):([0-5]\d):([0-5]\d))", t) ]
for f in find:
    print(f)


['23:15:59', 'zsd32:12:23', '  17:12:32  23:21:54', '12_12_12 ', 'awtgdvgs5eyrsg   00:02:03 ', 'zsgfb:ds:se', '   16:12:45']
Строка 0, позиция 0 : найдено '23:15:59'
Строка 2, позиция 2 : найдено '17:12:32'
Строка 2, позиция 12 : найдено '23:21:54'
Строка 4, позиция 17 : найдено '00:02:03'
Строка 6, позиция 3 : найдено '16:12:45'


5. Введите с клавиатуры текст. Программно найдите в нем и выведите
отдельно все слова, которые начинаются с большого латинского
символа (от A до Z) и заканчиваются 2 или 4 цифрами, например
«Petr93», «Johnny70», «Service2002». Используйте регулярные
выражения.

In [28]:
text = input('Введите текст : ')
# text = 'Petr93 Tb12345 GHB1 THsc123 P1234'
find = re.findall(r'([A-Z][a-zA-Z]*(?:\d\d|\d{4}))(?:\s|\b|$)',text)
find

['Petr93', 'P1234']

6. Напишите скрипт `reorganize.py`, который в директории `--source` создает две директории: Archive и Small. В первую директорию помещаются файлы с датой изменения, отличающейся от текущей даты на количество дней более параметра `--days` (т.е. относительно старые файлы). Во вторую – все файлы размером меньше параметра `--size` байт.

Каждая директория должна создаваться только в случае, если найден хотя бы один файл, который должен быть в нее помещен.
Пример вызова:
```
reorganize --source "C:\TestDir" --days 2 --size 4096```


In [5]:
# start reorganize.py with commandline or

cmds = [sys.executable,'reorganize.py', '--source', "D:\Test", '--days','2', '--size','25']
output = subprocess.check_output(cmds)
print(output.decode('cp866'))

Namespace(days=2, size=25, source='D:\\Test')
Copy_of_[homework]2_python_intro.ipynb t: 2020-04-14 18:01:33.583008 s: 10616b delta: 1
Copy_of_[seminar,prodv]2_Numpy.ipynb t: 2020-04-14 18:01:33.601562 s: 259238b delta: 1
labs.rar t: 2020-04-14 18:09:38.140625 s: 67489b delta: 1
Python Lab1 (5) (6).ipynb t: 2020-04-14 18:01:33.624023 s: 21943b delta: 1
[homework,_base]3_numpy.ipynb t: 2020-04-14 18:01:33.633789 s: 21317b delta: 1
[seminar]python_basics.ipynb t: 2020-04-14 18:01:33.647461 s: 86419b delta: 1
[seminar]tools.ipynb t: 2020-04-14 18:01:33.658203 s: 23856b delta: 1
[sem_solution]python_basics.ipynb t: 2020-04-14 18:01:33.670898 s: 132155b delta: 1



7. Написать скрипт trackmix.py, который формирует обзорный трек-микс альбома (попурри из коротких фрагментов mp3-файлов в пользовательской директории). Для манипуляций со звуковыми файлами можно использовать сторонние утилиты, например, FFmpeg.

Пример вызова и работы скрипта:
```
trackmix --source "C:\Muz\Album" --count 5 --frame 15 -l -e
--- processing file 1: 01 - Intro.mp3
--- processing file 2: 02 - Outro.mp3
--- done!
```

Параметры скрипта:
- `--source` (`-s`) – имя рабочей директории с треками, обязателен;
- `--destination` (`-d`) – имя выходного файла, необязателен (если не указан, то имя выходного файла – mix.mp3 в директории source);
- `--count` (`-c`) – количество файлов в "нарезке", необязателен (если он не указан, то перебираются все mp3-файлы в директории source);
- `--frame` (`-f`) – количество секунд на каждый файл, необязателен (если не указан, скрипт вырезает по 10 секунд из произвольного участка каждого файла);
- `--log` (`-l`) – необязательный ключ (если этот ключ указывается, то скрипт должен выводить на консоль лог процесса обработки файлов, как в примере);
- `--extended` (`-e`) – необязательный ключ (если этот ключ указывается, то каждый фрагмент попурри начинается и завершается небольшим `fade in/fade out`)

In [ ]:
# start trackmix.py with commandline or

In [9]:
cmds = [sys.executable,'trackmix.py','-s',r'D:\music','-d','mixed.mp3','-c','5','-l','-e']
output = subprocess.check_output(cmds)
print(output.decode('cp866'))

--- processing file 1: 01. Aerosmith - Dream On [04.26].mp3
--- processing file 2: 02. Apocalyptica - Hope [03.25].mp3
--- processing file 3: 03. Avantasia - In Quest For [03.53].mp3
--- processing file 4: 04. Avicii, Sandro Cavazza - Without You [03.01].mp3
--- processing file 5: 05. Black Lab - This Night [03.29].mp3
--- mergering the frames...
--- deleting temporary files...
--- done

